<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Basic Time Series Plotting</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="http://matplotlib.org/_images/date_demo.png" alt="METAR" style="height: 300px;"></div>


## Overview:

* **Teaching:** 45 minutes
* **Exercises:** 30 minutes

### Questions
1. How can we obtain buoy data from the NDBC?
1. How are plots created in Python?
1. What features does Matplotlib have for improving our time series plots?
1. How can multiple y-axes be used in a single plot?

### Objectives
1. <a href="#loaddata">Obtaining data</a>
1. <a href="#basictimeseries">Basic timeseries plotting</a>
1. <a href="#multiy">Multiple y-axes</a>

<a name="loaddata"></a>
## Obtaining Data
To learn about time series analysis, we first need to find some data and get it into Python. In this case we're going to use data from the [National Data Buoy Center](http://www.ndbc.noaa.gov). We'll use the [pandas](http://pandas.pydata.org) library for our data subset and manipulation operations after obtaining the data with siphon. 

Each buoy has many types of data availabe, you can read all about it in the [NDBC Web Data Guide](https://www.ndbc.noaa.gov/docs/ndbc_web_data_guide.pdf). There is a mechanism in siphon to see which data types are available for a given buoy.

In [ ]:
from siphon.simplewebservice.ndbc import NDBC

data_types = NDBC.buoy_data_types('46042')
print(data_types)

In this case, we'll just stick with the standard meteorological data. The "realtime" data from NDBC contains approximately 45 days of data from each buoy. We'll retreive that record for buoy 51002 and then do some cleaning of the data. 

In [ ]:
df = NDBC.realtime_observations('46042')

In [ ]:
df.tail()

Let's get rid of the columns with all missing data. We could use the `drop` method and manually name all of the columns, but that would require us to know which are all `NaN` and that sounds like manual labor - something that programmers hate. Pandas has the `dropna` method that allows us to drop rows or columns where any or all values are `NaN`. In this case, let's drop all columns with all `NaN` values.

In [ ]:
df = df.dropna(axis='columns', how='all')

In [ ]:
df.head()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Use the realtime_observations method to retreive supplemental data for buoy 41002. **Note** assign the data to something other that df or you'll have to rerun the data download cell above. We suggest using the name supl_obs.</li>
    </ul>
</div>

In [ ]:
# Your code goes here
# supl_obs =

#### Solution

In [ ]:
# %load solutions/get_obs.py

Finally, we need to trim down the data. The file contains 45 days worth of observations. Let's look at the last week's worth of data.

In [ ]:
import pandas as pd
idx = df.time >= (pd.Timestamp.utcnow() - pd.Timedelta(days=7))
df = df[idx]
df.head()

We're almost ready, but now the index column is not that meaningful. It starts at a non-zero row, which is fine with our initial file, but let's re-zero the index so we have a nice clean data frame to start with.

In [ ]:
df.reset_index(drop=True, inplace=True)
df.head()

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="basictimeseries"></a>
## Basic Timeseries Plotting

Matplotlib is a python 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms. We're going to learn the basics of creating timeseries plots with matplotlib by plotting buoy wind, gust, temperature, and pressure data.

In [ ]:
# Convention for import of the pyplot interface
import matplotlib.pyplot as plt

# Set-up to have matplotlib use its support for notebook inline plots
%matplotlib inline

We'll start by plotting the windspeed observations from the buoy.

In [ ]:
plt.rc('font', size=12)
fig, ax = plt.subplots(figsize=(10, 6))

# Specify how our lines should look
ax.plot(df.time, df.wind_speed, color='tab:orange', label='Windspeed')

# Same as above
ax.set_xlabel('Time')
ax.set_ylabel('Speed (m/s)')
ax.set_title('Buoy Wind Data')
ax.grid(True)
ax.legend(loc='upper left');

Our x axis labels look a little crowded - let's try only labeling each day in our time series.

In [ ]:
# Helpers to format and locate ticks for dates
from matplotlib.dates import DateFormatter, DayLocator

# Set the x-axis to do major ticks on the days and label them like '07/20'
ax.xaxis.set_major_locator(DayLocator())
ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))

fig

Now we can add wind gust speeds to the same plot as a dashed yellow line.

In [ ]:
# Use linestyle keyword to style our plot
ax.plot(df.time, df.wind_gust, color='tab:olive', linestyle='--',
        label='Wind Gust')
# Redisplay the legend to show our new wind gust line
ax.legend(loc='upper left')

fig

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li>Create your own figure and axes (<code>myfig, myax = plt.subplots(figsize=(10, 6))</code>) which plots temperature.</li>
    <li>Change the x-axis major tick labels to display the shortened month and date (i.e. 'Sep DD' where DD is the day number). Look at the
        <a href="https://docs.python.org/3.6/library/datetime.html#strftime-and-strptime-behavior">
            table of formatters</a> for help.
    <li>Make sure you include a legend and labels!</li>
    <li><b>BONUS:</b> try changing the <code>linestyle</code>, e.g., a blue dashed line.</li>
    </ul>
</div>

In [ ]:
# Your code goes here


#### Solution
<div class="alert alert-info">
    <b>Tip</b>:
     If your figure goes sideways as you try multiple things, try running the notebook up to this point again
     by using the Cell -> Run All Above option in the menu bar.
</div>

In [ ]:
# %load solutions/basic_plot.py

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="multiy"></a>
## Multiple y-axes
What if we wanted to plot another variable in vastly different units on our plot? <br/>
Let's return to our wind data plot and add pressure.

In [ ]:
# plot pressure data on same figure
ax.plot(df.time, df.pressure, color='black', label='Pressure')
ax.set_ylabel('Pressure')

ax.legend(loc='upper left')

fig

That is less than ideal. We can't see detail in the data profiles! We can create a twin of the x-axis and have a secondary y-axis on the right side of the plot. We'll create a totally new figure here.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
axb = ax.twinx()

# Same as above
ax.set_xlabel('Time')
ax.set_ylabel('Speed (m/s)')
ax.set_title('Buoy Data')
ax.grid(True)

# Plotting on the first y-axis
ax.plot(df.time, df.wind_speed, color='tab:orange', label='Windspeed')
ax.plot(df.time, df.wind_gust, color='tab:olive', linestyle='--', label='Wind Gust')
ax.legend(loc='upper left');

# Plotting on the second y-axis
axb.set_ylabel('Pressure (hPa)')
axb.plot(df.time, df.pressure, color='black', label='pressure')

ax.xaxis.set_major_locator(DayLocator())
ax.xaxis.set_major_formatter(DateFormatter('%b %d'))


We're closer, but the data are plotting over the legend and not included in the legend. That's because the legend is associated with our primary y-axis. We need to append that data from the second y-axis.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
axb = ax.twinx()

# Same as above
ax.set_xlabel('Time')
ax.set_ylabel('Speed (m/s)')
ax.set_title('Buoy 41056 Wind Data')
ax.grid(True)

# Plotting on the first y-axis
ax.plot(df.time, df.wind_speed, color='tab:orange', label='Windspeed')
ax.plot(df.time, df.wind_gust, color='tab:olive', linestyle='--', label='Wind Gust')

# Plotting on the second y-axis
axb.set_ylabel('Pressure (hPa)')
axb.plot(df.time, df.pressure, color='black', label='pressure')

ax.xaxis.set_major_locator(DayLocator())
ax.xaxis.set_major_formatter(DateFormatter('%b %d'))

# Handling of getting lines and labels from all axes for a single legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = axb.get_legend_handles_labels()
axb.legend(lines + lines2, labels + labels2, loc='upper left');

<div class="alert alert-success">
    <b>EXERCISE</b>:
    Create your own plot that has the following elements:
     <ul>
    <li>A blue line representing the wave height measurements.</li>
    <li>A green line representing wind speed on a secondary y-axis</li>
    <li>Proper labels/title.</li>
    <li>**Bonus**: Make the wave height data plot as points only with no line. Look at the documentation for the linestyle and marker arguments.</li>
    </ul>
</div>

In [ ]:
# Your code goes here


#### Solution

In [ ]:
# %load solutions/adv_plot.py

<a href="#top">Top</a>
<hr style="height:2px;">